In [5]:
from sklearn.datasets import load_boston
import numpy as np
import matplotlib.pyplot as plt
from LinearRegression import LinearRegression
from model_selection import train_test_split 

In [6]:
data = load_boston()
data_X = data.data
data_y = data.target

data_X = data_X[data_y<50]
data_y = data_y[data_y<50]

In [8]:
X_train,y_train,X_test,y_test = train_test_split(data_X,data_y,seed=666)

### 训练模型 ：使用梯度下降法

In [9]:
lr1 = LinearRegression()

In [13]:
lr1.fit_gd(X_train,y_train,eta=0.000001)

LinearRegression()  with normal_equation

In [14]:
lr1.score(X_test,y_test)

0.27556634853389228

#### 效果十分差 分析原因：可能由于eta十分小 经过默认1e4的迭代之后可能未收敛到局部最优点
### 改进:

In [18]:
%%time
lr2 = LinearRegression()
lr2.fit_gd(X_train,y_train,eta = 0.000001,n_iters=1e6)

Wall time: 46.8 s


In [19]:
lr2.score(X_test,y_test)

0.75418523539807636

### 有些提升但是 没有到全局最优点（从正规方程的方法score能看出来）
## 如下：



In [21]:
%%time
lr3 = LinearRegression()
lr3.fit_normal(X_train,y_train )

Wall time: 408 ms


In [22]:
lr3.score(X_test,y_test)

0.81298026026583592

### 接着改进：我想到了规一化数据（减少因数据各个特征的规模不同对fit模型造成的影响）

In [23]:
from my_preprocessing import my_StandardScaler

In [24]:
mysS = my_StandardScaler()

In [26]:
mysS.fit(X_train)

In [28]:
X_train_standard = mysS.transform(X_train)

In [33]:
%%time
lr4 = LinearRegression()
lr4.fit_gd(X_train_standard,y_train)

Wall time: 227 ms


In [34]:
X_test_Standard = mysS.transform(X_test)
lr4.score(X_test_Standard,y_test)

0.81298806201222351

### 如上 在默认参数下 eta = 0.01 时 依旧收敛到了局部最优点

# 对比正规方程  好处：
在n比较大的时候 梯度下降会很快   normal equation时间复杂度O（N^3）

In [37]:
#制作一组数据对比~
m = 1000
n = 10000

big_X = np.random.normal(size=(m, n))

true_theta = np.random.uniform(0.0, 100.0, size=n+1)

big_y = big_X.dot(true_theta[1:]) + true_theta[0] + np.random.normal(0., 10., size=m)

In [38]:
big_reg1 = LinearRegression()
%time big_reg1.fit_normal(big_X, big_y)

Wall time: 27.2 s


LinearRegression()  with normal_equation

In [39]:
big_reg2 = LinearRegression()
%time big_reg2.fit_gd(big_X, big_y)

Wall time: 2.32 s


LinearRegression()  with normal_equation

### 以上时间 说明了一切！